In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from sqlalchemy import text

In [2]:
DB_USER = "root"         
DB_PASS = "YourPassword"     
DB_HOST = "localhost"
DB_PORT = "3306"
DB_NAME = "e_commerce_project"

In [3]:
engine = create_engine(f"mysql+pymysql://{DB_USER}:{DB_PASS}@{DB_HOST}/{DB_NAME}")

In [4]:
print("📥 Loading tables from MySQL...")
customers = pd.read_sql("SELECT * FROM customers", engine)
orders = pd.read_sql("SELECT * FROM orders", engine)
order_items = pd.read_sql("SELECT * FROM order_items", engine)
payments = pd.read_sql("SELECT * FROM payments", engine)
products = pd.read_sql("SELECT * FROM products", engine)

📥 Loading tables from MySQL...


In [5]:
print("🧹 Cleaning data...")

date_cols = [
    "order_purchase_timestamp",
    "order_approved_at",
    "order_delivered_carrier_date",
    "order_delivered_customer_date",
    "order_estimated_delivery_date"
]
for col in date_cols:
    if col in orders.columns:
        orders[col] = pd.to_datetime(orders[col], errors="coerce")

customers.drop_duplicates(subset=["customer_id"], inplace=True)

payments = payments.dropna(subset=["order_id", "payment_value"])


🧹 Cleaning data...


In [6]:
print("🔄 Merging tables...")

olist_clean = (
    orders
    .merge(payments, on="order_id", how="left")
    .merge(order_items, on="order_id", how="left")
    .merge(products, on="product_id", how="left")
    .merge(customers, on="customer_id", how="left")
)

olist_clean = olist_clean[[
    "order_id", "customer_id", "customer_city", "customer_state",
    "order_status", "order_purchase_timestamp", "order_delivered_customer_date",
    "payment_type", "payment_installments", "payment_value",
    "product_id", "product_category_name", "price", "freight_value"
]]

🔄 Merging tables...


In [7]:
olist_clean["order_id"] = olist_clean["order_id"].astype(str)
olist_clean["customer_id"] = olist_clean["customer_id"].astype(str)
olist_clean["product_id"] = olist_clean["product_id"].astype(str)

# Save to MySQL with dtype mapping
from sqlalchemy.types import String

olist_clean.to_sql(
    "olist_clean",
    engine,
    if_exists="replace",
    index=False,
    dtype={
        "order_id": String(50),
        "customer_id": String(50),
        "product_id": String(50)
    }
)
print("💾 Saving cleaned table to MySQL...")

💾 Saving cleaned table to MySQL...


In [8]:
with engine.begin() as conn:
    conn.execute(text("ALTER TABLE olist_clean ADD INDEX idx_order_id (order_id)"))
    conn.execute(text("ALTER TABLE olist_clean ADD INDEX idx_customer_id (customer_id)"))
    conn.execute(text("ALTER TABLE olist_clean ADD INDEX idx_product_id (product_id)"))

print("✅ Data cleaning and merging complete! The cleaned table is saved as 'olist_clean' in the MySQL database.")  

✅ Data cleaning and merging complete! The cleaned table is saved as 'olist_clean' in the MySQL database.
